In [1]:
import sys
sys.path.append('/dccstor/hoo-misha-1/wilds/wilds/examples')
sys.path.append('/dccstor/hoo-misha-1/wilds/WOODS')
sys.path.append('/dccstor/hoo-misha-1/wilds/wilds')

import os

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn

from transformers import DistilBertModel, DistilBertTokenizerFast
from models.bert.distilbert import DistilBertClassifier, DistilBertFeaturizer
from configs.datasets import dataset_defaults

import wilds
from wilds.common.data_loaders import get_train_loader, get_eval_loader
from wilds.common.grouper import CombinatorialGrouper
from wilds.datasets.unlabeled.wilds_unlabeled_dataset import WILDSPseudolabeledSubset

from utils import set_seed, Logger, BatchLogger, log_config, ParseKwargs, load, initialize_wandb, log_group_data, parse_bool, get_model_prefix, move_to
from train import train, evaluate, infer_predictions,run_epoch
from algorithms.initializer import initialize_algorithm, infer_d_out
from transforms import initialize_transform

from models.initializer import initialize_model
from configs.utils import populate_defaults
import configs.supported as supported

import torch.multiprocessing

import torchvision.transforms as transforms

from examples.transforms import initialize_bert_transform

from tqdm import tqdm
import argparse
import copy
import re
import psutil
from collections import defaultdict

# Initialize Wilds Config

In [2]:
''' Arg defaults are filled in according to examples/configs/ '''
parser = argparse.ArgumentParser()

# Required arguments
parser.add_argument('-d', '--dataset', choices=wilds.supported_datasets, required=True)
parser.add_argument('--algorithm', required=True, choices=supported.algorithms)
parser.add_argument('--root_dir', required=True,
                    help='The directory where [dataset]/data can be found (or should be downloaded to, if it does not exist).')

# Dataset
parser.add_argument('--split_scheme', help='Identifies how the train/val/test split is constructed. Choices are dataset-specific.')
parser.add_argument('--dataset_kwargs', nargs='*', action=ParseKwargs, default={},
                    help='keyword arguments for dataset initialization passed as key1=value1 key2=value2')
parser.add_argument('--download', default=False, type=parse_bool, const=True, nargs='?',
                    help='If true, tries to download the dataset if it does not exist in root_dir.')
parser.add_argument('--frac', type=float, default=1.0,
                    help='Convenience parameter that scales all dataset splits down to the specified fraction, for development purposes. Note that this also scales the test set down, so the reported numbers are not comparable with the full test set.')
parser.add_argument('--version', default=None, type=str, help='WILDS labeled dataset version number.')

# Unlabeled Dataset
parser.add_argument('--unlabeled_split', default=None, type=str, choices=wilds.unlabeled_splits,  help='Unlabeled split to use. Some datasets only have some splits available.')
parser.add_argument('--unlabeled_version', default=None, type=str, help='WILDS unlabeled dataset version number.')
parser.add_argument('--use_unlabeled_y', default=False, type=parse_bool, const=True, nargs='?', 
                    help='If true, unlabeled loaders will also the true labels for the unlabeled data. This is only available for some datasets. Used for "fully-labeled ERM experiments" in the paper. Correct functionality relies on CrossEntropyLoss using ignore_index=-100.')

# Loaders
parser.add_argument('--loader_kwargs', nargs='*', action=ParseKwargs, default={})
parser.add_argument('--unlabeled_loader_kwargs', nargs='*', action=ParseKwargs, default={})
parser.add_argument('--train_loader', choices=['standard', 'group'])
parser.add_argument('--uniform_over_groups', type=parse_bool, const=True, nargs='?', help='If true, sample examples such that batches are uniform over groups.')
parser.add_argument('--distinct_groups', type=parse_bool, const=True, nargs='?', help='If true, enforce groups sampled per batch are distinct.')
parser.add_argument('--n_groups_per_batch', type=int)
parser.add_argument('--unlabeled_n_groups_per_batch', type=int)
parser.add_argument('--batch_size', type=int)
parser.add_argument('--unlabeled_batch_size', type=int)
parser.add_argument('--eval_loader', choices=['standard'], default='standard')
parser.add_argument('--gradient_accumulation_steps', type=int, default=1, help='Number of batches to process before stepping optimizer and schedulers. If > 1, we simulate having a larger effective batch size (though batchnorm behaves differently).')

# Model
parser.add_argument('--model', choices=supported.models)
parser.add_argument('--model_kwargs', nargs='*', action=ParseKwargs, default={},
                    help='keyword arguments for model initialization passed as key1=value1 key2=value2')
parser.add_argument('--noisystudent_add_dropout', type=parse_bool, const=True, nargs='?', help='If true, adds a dropout layer to the student model of NoisyStudent.')
parser.add_argument('--noisystudent_dropout_rate', type=float)
parser.add_argument('--pretrained_model_path', default=None, type=str, help='Specify a path to pretrained model weights')
parser.add_argument('--load_featurizer_only', default=False, type=parse_bool, const=True, nargs='?', help='If true, only loads the featurizer weights and not the classifier weights.')

# NoisyStudent-specific loading
parser.add_argument('--teacher_model_path', type=str, help='Path to NoisyStudent teacher model weights. If this is defined, pseudolabels will first be computed for unlabeled data before anything else runs.')

# Transforms
parser.add_argument('--transform', choices=supported.transforms)
parser.add_argument('--additional_train_transform', choices=supported.additional_transforms, help='Optional data augmentations to layer on top of the default transforms.')
parser.add_argument('--target_resolution', nargs='+', type=int, help='The input resolution that images will be resized to before being passed into the model. For example, use --target_resolution 224 224 for a standard ResNet.')
parser.add_argument('--resize_scale', type=float)
parser.add_argument('--max_token_length', type=int)
parser.add_argument('--randaugment_n', type=int, help='Number of RandAugment transformations to apply.')

# Objective
parser.add_argument('--loss_function', choices=supported.losses)
parser.add_argument('--loss_kwargs', nargs='*', action=ParseKwargs, default={},
                    help='keyword arguments for loss initialization passed as key1=value1 key2=value2')

# Algorithm
parser.add_argument('--groupby_fields', nargs='+')
parser.add_argument('--group_dro_step_size', type=float)
parser.add_argument('--coral_penalty_weight', type=float)
parser.add_argument('--wasserstein_blur', type=float, default=0.0001)
parser.add_argument('--dann_penalty_weight', type=float)
parser.add_argument('--dann_classifier_lr', type=float)
parser.add_argument('--dann_featurizer_lr', type=float)
parser.add_argument('--dann_discriminator_lr', type=float)
parser.add_argument('--afn_penalty_weight', type=float)
parser.add_argument('--safn_delta_r', type=float)
parser.add_argument('--hafn_r', type=float)
parser.add_argument('--use_hafn', default=False, type=parse_bool, const=True, nargs='?')
parser.add_argument('--irm_lambda', type=float)
parser.add_argument('--irm_penalty_anneal_iters', type=int)
parser.add_argument('--self_training_lambda', type=float)
parser.add_argument('--self_training_threshold', type=float)
parser.add_argument('--pseudolabel_T2', type=float, help='Percentage of total iterations at which to end linear scheduling and hold lambda at the max value')
parser.add_argument('--soft_pseudolabels', default=False, type=parse_bool, const=True, nargs='?')
parser.add_argument('--algo_log_metric')
parser.add_argument('--process_pseudolabels_function', choices=supported.process_pseudolabels_functions)

# Model selection
parser.add_argument('--val_metric')
parser.add_argument('--val_metric_decreasing', type=parse_bool, const=True, nargs='?')

# Optimization
parser.add_argument('--n_epochs', type=int)
parser.add_argument('--optimizer', choices=supported.optimizers)
parser.add_argument('--lr', type=float)
parser.add_argument('--weight_decay', type=float)
parser.add_argument('--max_grad_norm', type=float)
parser.add_argument('--optimizer_kwargs', nargs='*', action=ParseKwargs, default={},
                    help='keyword arguments for optimizer initialization passed as key1=value1 key2=value2')

# Scheduler
parser.add_argument('--scheduler', choices=supported.schedulers)
parser.add_argument('--scheduler_kwargs', nargs='*', action=ParseKwargs, default={},
                    help='keyword arguments for scheduler initialization passed as key1=value1 key2=value2')
parser.add_argument('--scheduler_metric_split', choices=['train', 'val'], default='val')
parser.add_argument('--scheduler_metric_name')

# Evaluation
parser.add_argument('--process_outputs_function', choices = supported.process_outputs_functions)
parser.add_argument('--evaluate_all_splits', type=parse_bool, const=True, nargs='?', default=True)
parser.add_argument('--eval_splits', nargs='+', default=[])
parser.add_argument('--eval_only', type=parse_bool, const=True, nargs='?', default=False)
parser.add_argument('--eval_epoch', default=None, type=int, help='If eval_only is set, then eval_epoch allows you to specify evaluating at a particular epoch. By default, it evaluates the best epoch by validation performance.')

# Misc
parser.add_argument('--device', type=int, nargs='+', default=[0])
parser.add_argument('--seed', type=int, default=0)
parser.add_argument('--log_dir', default='./logs')
parser.add_argument('--log_every', default=50, type=int)
parser.add_argument('--save_step', type=int)
parser.add_argument('--save_best', type=parse_bool, const=True, nargs='?', default=True)
parser.add_argument('--save_last', type=parse_bool, const=True, nargs='?', default=True)
parser.add_argument('--save_pred', type=parse_bool, const=True, nargs='?', default=True)
parser.add_argument('--no_group_logging', type=parse_bool, const=True, nargs='?')
parser.add_argument('--progress_bar', type=parse_bool, const=True, nargs='?', default=False)
parser.add_argument('--resume', type=parse_bool, const=True, nargs='?', default=False, help='Whether to resume from the most recent saved model in the current log_dir.')

# Weights & Biases
parser.add_argument('--use_wandb', type=parse_bool, const=True, nargs='?', default=False)
parser.add_argument('--wandb_api_key_path', type=str,
                    help="Path to Weights & Biases API Key. If use_wandb is set to True and this argument is not specified, user will be prompted to authenticate.")
parser.add_argument('--wandb_kwargs', nargs='*', action=ParseKwargs, default={},
                    help='keyword arguments for wandb.init() passed as key1=value1 key2=value2')

# BREEDS
parser.add_argument('--breeds', type=parse_bool, const=True, nargs='?', default=False) 

_StoreAction(option_strings=['--breeds'], dest='breeds', nargs='?', const=True, default=False, type=<function parse_bool at 0x1543cf47e9d0>, choices=None, help=None, metavar=None)

In [3]:
def update_config(parser, dataset, algorithm, model_path):
    global config
    print(f'|   Updating config to use algorithm {algorithm} and pretrained model path {model_path}')
    config = parser.parse_args((f'--dataset {dataset} '
                            f'--algorithm {algorithm} ' 
                            '--root_dir /dccstor/hoo-misha-1/wilds/wilds/data '
                            f'--pretrained_model_path {model_path} '
                            f'--wasserstein_blur 0.0001 '
                            #'--eval_only '
                            #'--model_kwargs ignore_mismatched_sizes=True ' 
                            '--evaluate_all_splits False '
                            #'--use_wandb '
                            ).split())
    config = populate_defaults(config)

In [4]:
update_config(parser, 'iwildcam', 'DANN', '/dccstor/hoo-misha-1/wilds/wilds/pretrained/breeds/breeds_DANN.pth')

|   Updating config to use algorithm DANN and pretrained model path /dccstor/hoo-misha-1/wilds/wilds/pretrained/breeds/breeds_DANN.pth


In [5]:
    
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    if len(config.device) > device_count:
        raise ValueError(f"Specified {len(config.device)} devices, but only {device_count} devices found.")

    config.use_data_parallel = len(config.device) > 1
    device_str = ",".join(map(str, config.device))
    os.environ["CUDA_VISIBLE_DEVICES"] = device_str
    config.device = torch.device("cuda")
else:
    config.use_data_parallel = False
    config.device =torch.device("cpu")

# Initialize logs
if os.path.exists(config.log_dir) and config.resume:
    resume=True
    mode='a'
elif os.path.exists(config.log_dir) and config.eval_only:
    resume=False
    mode='a'
else:
    resume=False
    mode='w'

if not os.path.exists(config.log_dir):
    os.makedirs(config.log_dir)
logger = Logger(os.path.join(config.log_dir, 'log.txt'), mode)


In [6]:
# Data
full_dataset = wilds.get_dataset(
    dataset=config.dataset,
    version=config.version,
    root_dir=config.root_dir,
    download=config.download,
    split_scheme=config.split_scheme,
    **config.dataset_kwargs)

# Transforms & data augmentations for labeled dataset
# To modify data augmentation, modify the following code block.
# If you want to use transforms that modify both `x` and `y`,
# set `do_transform_y` to True when initializing the `WILDSSubset` below.
train_transform = initialize_transform(
    transform_name=config.transform,
    config=config,
    dataset=full_dataset,
    additional_transform_name=config.additional_train_transform,
    is_training=True)
eval_transform = initialize_transform(
    transform_name=config.transform,
    config=config,
    dataset=full_dataset,
    is_training=False)

# Configure unlabeled datasets
unlabeled_dataset = None
if config.unlabeled_split is not None:
    split = config.unlabeled_split
    full_unlabeled_dataset = wilds.get_dataset(
        dataset=config.dataset,
        version=config.unlabeled_version,
        root_dir=config.root_dir,
        download=config.download,
        unlabeled=True,
        **config.dataset_kwargs
    )
    train_grouper = CombinatorialGrouper(
        dataset=[full_dataset, full_unlabeled_dataset],
        groupby_fields=config.groupby_fields
    )

    # Transforms & data augmentations for unlabeled dataset
    if config.algorithm == "FixMatch":
        # For FixMatch, we need our loader to return batches in the form ((x_weak, x_strong), m)
        # We do this by initializing a special transform function
        unlabeled_train_transform = initialize_transform(
            config.transform, config, full_dataset, is_training=True, additional_transform_name="fixmatch"
        )
    else:
        # Otherwise, use the same data augmentations as the labeled data.
        unlabeled_train_transform = train_transform

    if config.algorithm == "NoisyStudent":
        # For Noisy Student, we need to first generate pseudolabels using the teacher
        # and then prep the unlabeled dataset to return these pseudolabels in __getitem__
        print("Inferring teacher pseudolabels for Noisy Student")
        assert config.teacher_model_path is not None
        if not config.teacher_model_path.endswith(".pth"):
            # Use the best model
            config.teacher_model_path = os.path.join(
                config.teacher_model_path,  f"{config.dataset}_seed:{config.seed}_epoch:best_model.pth"
            )

        d_out = infer_d_out(full_dataset, config)
        teacher_model = initialize_model(config, d_out).to(config.device)
        load(teacher_model, config.teacher_model_path, device=config.device)
        # Infer teacher outputs on weakly augmented unlabeled examples in sequential order
        weak_transform = initialize_transform(
            transform_name=config.transform,
            config=config,
            dataset=full_dataset,
            is_training=True,
            additional_transform_name="weak"
        )
        unlabeled_split_dataset = full_unlabeled_dataset.get_subset(split, transform=weak_transform, frac=config.frac)
        sequential_loader = get_eval_loader(
            loader=config.eval_loader,
            dataset=unlabeled_split_dataset,
            grouper=train_grouper,
            batch_size=config.unlabeled_batch_size,
            **config.unlabeled_loader_kwargs
        )
        teacher_outputs = infer_predictions(teacher_model, sequential_loader, config)
        teacher_outputs = move_to(teacher_outputs, torch.device("cpu"))
        unlabeled_split_dataset = WILDSPseudolabeledSubset(
            reference_subset=unlabeled_split_dataset,
            pseudolabels=teacher_outputs,
            transform=unlabeled_train_transform,
            collate=full_dataset.collate,
        )
        teacher_model = teacher_model.to(torch.device("cpu"))
        del teacher_model
    else:
        unlabeled_split_dataset = full_unlabeled_dataset.get_subset(
            split, 
            transform=unlabeled_train_transform, 
            frac=config.frac, 
            load_y=config.use_unlabeled_y
        )

    unlabeled_dataset = {
        'split': split,
        'name': full_unlabeled_dataset.split_names[split],
        'dataset': unlabeled_split_dataset
    }
    unlabeled_dataset['loader'] = get_train_loader(
        loader=config.train_loader,
        dataset=unlabeled_dataset['dataset'],
        batch_size=config.unlabeled_batch_size,
        uniform_over_groups=config.uniform_over_groups,
        grouper=train_grouper,
        distinct_groups=config.distinct_groups,
        n_groups_per_batch=config.unlabeled_n_groups_per_batch,
        **config.unlabeled_loader_kwargs
    )
else:
    train_grouper = CombinatorialGrouper(
        dataset=full_dataset,
        groupby_fields=config.groupby_fields
    )

# Configure labeled torch datasets (WILDS dataset splits)
wilds_datasets = defaultdict(dict)
for split in full_dataset.split_dict.keys():
    if split=='train':
        transform = train_transform
        verbose = True
    elif split == 'val':
        transform = eval_transform
        verbose = True
    else:
        transform = eval_transform
        verbose = False
    # Get subset
    wilds_datasets[split]['dataset'] = full_dataset.get_subset(
        split,
        frac=config.frac,
        transform=transform)

    if split == 'train':
        wilds_datasets[split]['loader'] = get_train_loader(
            loader=config.train_loader,
            dataset=wilds_datasets[split]['dataset'],
            batch_size=config.batch_size,
            uniform_over_groups=config.uniform_over_groups,
            grouper=train_grouper,
            distinct_groups=config.distinct_groups,
            n_groups_per_batch=config.n_groups_per_batch,
            **config.loader_kwargs)
    else:
        wilds_datasets[split]['loader'] = get_eval_loader(
            loader=config.eval_loader,
            dataset=wilds_datasets[split]['dataset'],
            grouper=train_grouper,
            batch_size=config.batch_size,
            **config.loader_kwargs)

    # Set fields
    wilds_datasets[split]['split'] = split
    wilds_datasets[split]['name'] = full_dataset.split_names[split]
    wilds_datasets[split]['verbose'] = verbose

    # Loggers
    wilds_datasets[split]['eval_logger'] = BatchLogger(
        os.path.join(config.log_dir, f'{split}_eval.csv'), mode=mode, use_wandb=config.use_wandb
    )
    wilds_datasets[split]['algo_logger'] = BatchLogger(
        os.path.join(config.log_dir, f'{split}_algo.csv'), mode=mode, use_wandb=config.use_wandb
    )


In [7]:
wilds_datasets['train']['dataset']._n_classes = 13

In [8]:
# Initialize algorithm & load pretrained weights if provided
algorithm = initialize_algorithm(
    config=config,
    datasets=wilds_datasets,
    train_grouper=train_grouper,
    unlabeled_dataset=unlabeled_dataset,
)

Initialized model with pretrained weights from /dccstor/hoo-misha-1/wilds/wilds/pretrained/breeds/breeds_DANN.pth previously trained for 11 epochs with previous val metric 0.9428203962574587 


In [9]:
from robustness.tools.breeds_helpers import make_entity13
from robustness.tools.breeds_helpers import ClassHierarchy
#from robustness import datasets
from scripts import datasets

# Initialize BREEDS

In [10]:
data_dir = '/dccstor/leonidka1/data/imagenet/ILSVRC/Data/CLS-LOC/'
info_dir = '/dccstor/hoo-misha-1/wilds/BREEDS-Benchmarks/imagenet_class_hierarchy/modified'
num_workers = 8
batch_size = 16

hier = ClassHierarchy(info_dir)

In [11]:
ret = make_entity13(info_dir, split="rand")
superclasses, subclass_split, label_map = ret

In [12]:
train_subclasses, test_subclasses = subclass_split
train_subclasses = np.array(train_subclasses)
test_subclasses = np.array(test_subclasses)

In [13]:
train_subclasses

array([[869, 399, 578, 735, 652, 610, 501, 445, 655, 842],
       [ 87,  92,  91, 137,  14, 145, 129, 135,  85,  21],
       [ 45,  42,  51,  47,  52,  63,  55,  35,  58,  33],
       [119,  72, 300,  75,  73, 317, 302, 120, 309, 313],
       [284, 350, 292, 344, 174, 149, 375, 258, 283, 170],
       [443, 552, 824, 728, 433, 514, 679, 518, 570, 638],
       [484, 814, 554, 914, 404, 510, 628, 871, 812, 403],
       [890, 681, 430, 590, 872, 745, 422, 664, 522, 416],
       [894, 861, 553, 669, 564, 431, 493, 559, 556, 789],
       [881, 695, 626, 822, 704, 499, 845, 398, 778, 512],
       [483, 442, 562, 727, 920, 460, 500, 663, 743, 900],
       [803, 867, 751, 791, 880, 670, 705, 654, 609, 757],
       [937, 987, 950, 943, 940, 942, 941, 938, 945, 952]])

In [14]:
len(np.unique(train_subclasses))

130

In [15]:
train_subdomains_subclasses = {}
test_subdomains_subclasses = {}
for i in range(train_subclasses.shape[1]):
    for subclass in train_subclasses[:,i]:
        train_subdomains_subclasses[subclass] = torch.tensor([i,i]).to('cpu')
    
for i in range(test_subclasses.shape[1]):
    for subclass in test_subclasses[:,i]:
        test_subdomains_subclasses[subclass] = torch.tensor([i,i]).to('cpu')

In [16]:
dataset_source = datasets.WILDSCustomImageNet(data_dir, train_subclasses, train_subdomains_subclasses)
loaders_source = dataset_source.make_loaders(num_workers, batch_size)
train_loader_source, val_loader_source = loaders_source

==> Preparing dataset custom_imagenet..


In [17]:
dataset_target = datasets.WILDSCustomImageNet(data_dir, test_subclasses, test_subdomains_subclasses)
loaders_target = dataset_target.make_loaders(num_workers, batch_size)
train_loader_target, val_loader_target = loaders_target

==> Preparing dataset custom_imagenet..


In [18]:
def prune_model():
    global config, wilds_datasets, train_grouper, unlabeled_dataset
    print(f'|   Pruning {config.model}')
    print("|   |   ", end = '')
    # Initialize algorithm & load pretrained weights if provided
    algorithm = initialize_algorithm(
        config=config,
        datasets=wilds_datasets,
        train_grouper=train_grouper,
    )
    if 'resnet' in config.model:
        algorithm.model = next(algorithm.model.children())
        for param in algorithm.model.parameters():
            param.requires_grad = False
        return algorithm.model
    elif 'densenet' in config.model:
        algorithm.model.classifier = nn.Identity(1024)
        for param in algorithm.model.parameters():
            param.requires_grad = False
        return algorithm.model
    raise Exception('New model, no pruning done')

# Replace WILDS dataset with BREEDS dataset

In [19]:
#model = prune_model()

In [20]:
algorithm = initialize_algorithm(
        config=config,
        datasets=wilds_datasets,
        train_grouper=train_grouper,
    )

Initialized model with pretrained weights from /dccstor/hoo-misha-1/wilds/wilds/pretrained/breeds/breeds_DANN.pth previously trained for 11 epochs with previous val metric 0.9428203962574587 


In [21]:
model = algorithm.model

In [22]:
#Deep CORAL
if config.algorithm == 'deepCORAL':
    model = next(model.children())
elif config.algorithm == 'wassersteindeepCORAL':
    model = next(model.children())
# ERM
elif config.algorithm == 'ERM':
    model.fc = nn.Identity(2048)
# DANN
elif config.algorithm == 'DANN':
    model.classifier = nn.Identity(2048)



In [23]:
features = np.array([None] * len(train_loader_target))
y = np.array([None] * len(train_loader_target))
metadata = np.array([None] * len(train_loader_target))

In [24]:
count = 0
for X_batch, y_batch , metadata_batch in tqdm(train_loader_target):
    #print(psutil.virtual_memory()[2])
    # if count % 100 == 0:
        # info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        # print("Device {}: {}, Memory : ({:.2f}% free): {}(total), {} (free), {} (used)".format(0, nvidia_smi.nvmlDeviceGetName(handle), 100*info.free/info.total, info.total, info.free, info.used))
        # print('CPU util', psutil.cpu_percent(4))
    try:
        features_batch = model(X_batch.to(config.device))
        if config.algorithm == 'DANN':
            features_batch = model(X_batch.to(config.device))[0]
        #print(features_batch.shape)
        if type(features_batch) is tuple:
            #print('is tup')
            features_batch = features_batch[0].detach().cpu().numpy()
        else:
            features_batch = features_batch.detach().cpu().numpy()
        y_batch = y_batch.detach().cpu().numpy()
        metadata_batch = metadata_batch.detach().cpu().numpy()
        #print('h')
        # if features is None:
        #     features = features_batch
        # else:
        #     features = np.vstack((features, features_batch))
        features[count] = features_batch
        #print(features_batch.device)
        # if y is None:
        #     y = y_batch
        # else:
        #     y = np.concatenate((y, y_batch))
        y[count] = y_batch
        metadata[count] = metadata_batch
        
        # if metadata is None:
        #     metadata = metadata_batch
        # else:
        #     metadata = np.vstack((metadata,metadata_batch))
        count += 1
    except Exception as e:
        print(e)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10475/10475 [07:31<00:00, 23.20it/s]


In [25]:
config.dataset = 'breeds'
split = 'test'

In [26]:
features = np.vstack(features)
y = np.concatenate(y)
metadata = np.vstack(metadata)
print(f'|   |   Features has shape {features.shape} and labels has shape {y.shape}')
save_path_base = f'/dccstor/hoo-misha-1/wilds/wilds/features/{config.dataset}/{config.algorithm}'
if not os.path.exists(save_path_base):
    os.makedirs(save_path_base)
features_save_path = f'{save_path_base}/{config.model}_{split}_features.npy'
labels_save_path = f'{save_path_base}/{config.model}_{split}_labels.npy'
metadata_save_path = f'{save_path_base}/{config.model}_{split}_metadata.npy'
np.save(features_save_path, features)
np.save(labels_save_path, y)
np.save(metadata_save_path, metadata)
print(f'|   |   |   Features saved to {features_save_path}')
print(f'|   |   |   Labels saved to {labels_save_path}')
print(f'|   |   |    Metadata saved to {metadata_save_path}')

|   |   Features has shape (167592, 2048) and labels has shape (167592,)
|   |   |   Features saved to /dccstor/hoo-misha-1/wilds/wilds/features/breeds/DANN/resnet50_test_features.npy
|   |   |   Labels saved to /dccstor/hoo-misha-1/wilds/wilds/features/breeds/DANN/resnet50_test_labels.npy
|   |   |    Metadata saved to /dccstor/hoo-misha-1/wilds/wilds/features/breeds/DANN/resnet50_test_metadata.npy
